In [196]:
import scipy.io as sio # python 讀取 .mat file
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Dropout
from keras.optimizers import Adam
from keras.utils import np_utils
from keras import backend as keras_backend
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices()) #check available device

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 9747801240775790832
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 2668979052289921627
physical_device_desc: "device: XLA_CPU device"
]


In [197]:
matData = sio.loadmat("fisheriris.mat") # 鳶尾花
#print(matData.items()) # 得知有哪些 items 

# iris(species, meas),(3 label, 4 attribute) 150 筆
irisSpecies = np.array(matData["species"]) # type(setosa, versicolor, virginica)
irisMeas = np.array(matData["meas"]) # sepal(花萼)長、寬, petal(花瓣)長、寬 cm
print(irisSpecies[0], irisMeas[0])
# MAT 格式問題，array[array(), array(), ...]
#print(irisSpecies[:, 0], irisMeas[0]) 各種 fail
#print([item[0][0] for item in irisSpecies])
irisSpecies = [item[0][0] for item in irisSpecies]

[array(['setosa'], dtype='<U6')] [5.1 3.5 1.4 0.2]


In [198]:
matData = sio.loadmat("ionosphere.mat") # 電離層
#print(matData.items())

# (X, Y),(34 attribute, 2 label) 351 筆
ionosphereX = np.array(matData["X"])
ionosphereY = np.array(matData["Y"])
print(ionosphereX[0], ionosphereY[0])
ionosphereY = [item[0][0] for item in ionosphereY]
#print(ionosphereY)

[ 1.       0.       0.99539 -0.05889  0.85243  0.02306  0.83398 -0.37708
  1.       0.0376   0.85243 -0.17755  0.59755 -0.44945  0.60536 -0.38223
  0.84356 -0.38542  0.58212 -0.32192  0.56971 -0.29674  0.36946 -0.47357
  0.56811 -0.51171  0.41078 -0.46168  0.21266 -0.3409   0.42267 -0.54487
  0.18641 -0.453  ] [array(['g'], dtype='<U1')]


In [199]:
# split data into (train, test) = (80%, 20%)
irisSpecies_train, irisSpecies_test, irisMeas_train, irisMeas_test = train_test_split(irisSpecies, irisMeas, test_size=0.2)
print("== iris ==")
print(irisMeas_train.shape)
print(irisMeas_test.shape)
#print(irisSpecies_train)

ionosphereX_train, ionosphereX_test, ionosphereY_train, ionosphereY_test = train_test_split(ionosphereX, ionosphereY, test_size=0.2)
print("== ionosphere ==")
print(ionosphereX_train.shape)
print(ionosphereX_test.shape)
#print(ionosphereY_train)

== iris ==
(120, 4)
(30, 4)
== ionosphere ==
(280, 34)
(71, 34)


In [200]:
"""
    Logistic Regression
"""

logreg = LogisticRegression(solver='lbfgs', multi_class='multinomial', max_iter=1e4)
print(logreg)

logreg.fit(irisMeas_train, irisSpecies_train)
predictIrisTrain = logreg.predict(irisMeas_train)
count = 0
for i in range(len(predictIrisTrain)):
    if predictIrisTrain[i] == irisSpecies_train[i]:
        count += 1
print("\n== iris ==")
print("iris train accurracy: %f %%" % ( count/len(predictIrisTrain) ) )
predictIris = logreg.predict(irisMeas_test)
count = 0
for i in range(len(predictIris)):
    if predictIris[i] == irisSpecies_test[i]:
        count += 1
print("iris test accurracy: %f %%" % ( count/len(predictIris) ) )


logreg.fit(ionosphereX_train, ionosphereY_train)
predictIonosphereTrain = logreg.predict(ionosphereX_train)
count = 0
for i in range(len(predictIonosphereTrain)):
    if predictIonosphereTrain[i] == ionosphereY_train[i]:
        count += 1
print("\n== ionosphere ==")
print("ionosphere accurracy: %f %%" % ( count/len(predictIonosphereTrain) ) )
predictIonosphere = logreg.predict(ionosphereX_test)
count = 0
for i in range(len(predictIonosphere)):
    if predictIonosphere[i] == ionosphereY_test[i]:
        count += 1
print("ionosphere accurracy: %f %%" % ( count/len(predictIonosphere) ) )

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=10000.0, multi_class='multinomial',
          n_jobs=None, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

== iris ==
iris train accurracy: 0.966667 %
iris test accurracy: 0.966667 %

== ionosphere ==
ionosphere accurracy: 0.925000 %
ionosphere accurracy: 0.887324 %


In [201]:
"""
    Neural Network
"""

#preprocessing data, turn label string -> vector
for i in range(len(irisSpecies_train)):
    if irisSpecies_train[i] == "setosa":
        irisSpecies_train[i] = 0
    elif irisSpecies_train[i] == "versicolor":
        irisSpecies_train[i] = 0
    else:
        irisSpecies_train[i] = 2
irisSpecies_train = np_utils.to_categorical(irisSpecies_train, 3)
for i in range(len(irisSpecies_test)):
    if irisSpecies_test[i] == "setosa":
        irisSpecies_test[i] = 0
    elif irisSpecies_test[i] == "versicolor":
        irisSpecies_test[i] = 0
    else:
        irisSpecies_test[i] = 2
irisSpecies_test = np_utils.to_categorical(irisSpecies_test, 3)

#print(ionosphereY_train)
for i in range(len(ionosphereY_train)):
    if ionosphereY_train[i] == "g":
        ionosphereY_train[i] = 0
    else:
        ionosphereY_train[i] = 1
ionosphereY_train = np_utils.to_categorical(ionosphereY_train, 2)
for i in range(len(ionosphereY_test)):
    if ionosphereY_test[i] == "g":
        ionosphereY_test[i] = 0
    else:
        ionosphereY_test[i] = 1
ionosphereY_test = np_utils.to_categorical(ionosphereY_test, 2)
#print(ionosphereY_train)
"""
irisSpecies_train
irisSpecies_test
irisMeas_train
irisMeas_tes


ionosphereX_train
ionosphereX_test
ionosphereY_train
ionosphereY_test
"""

'\nirisSpecies_train\nirisSpecies_test\nirisMeas_train\nirisMeas_tes\n\n\nionosphereX_train\nionosphereX_test\nionosphereY_train\nionosphereY_test\n'

In [206]:
# create the model
model = Sequential()

# Dense -> fully connected, (node num, input size, activation function)
model.add(Dense(32, input_dim=4, activation='relu'))
#input_shape=(784,) 等價於 input_dim=784 ，接收 n 個 784 input
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(3, activation='softmax'))


# compile the model
#model.compile(loss='categorical_crossentropy',
#              optimizer='sgd',
#              metrics=['accuracy'])            
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])            


# train the model on data
model.fit(irisMeas_train, irisSpecies_train,
          batch_size=1, # how many data to look at together
          epochs=20,    # how many epochs to run before stopping
          verbose=2,  # 訓練的詳細資料, 0=安靜, 1=進度條, 2=每輪一行
          validation_data=(irisMeas_test, irisSpecies_test)
         )

# test the trained model on the testing set
score = model.evaluate(irisMeas_train, irisSpecies_train, verbose=0) 
print("\n== Iris ==")
print('Train loss:', score[0]) #loss
print('Train accuracy:', score[1])
score = model.evaluate(irisMeas_test, irisSpecies_test, verbose=0) 

print('Test loss:', score[0]) #loss
print('Test accuracy:', score[1])

Train on 120 samples, validate on 30 samples
Epoch 1/20
 - 2s - loss: 1.0233 - acc: 0.4000 - val_loss: 0.7237 - val_acc: 0.9333
Epoch 2/20
 - 0s - loss: 0.6483 - acc: 0.7417 - val_loss: 0.5510 - val_acc: 0.7000
Epoch 3/20
 - 0s - loss: 0.6444 - acc: 0.6500 - val_loss: 0.4479 - val_acc: 0.7000
Epoch 4/20
 - 0s - loss: 0.5210 - acc: 0.7250 - val_loss: 0.3486 - val_acc: 0.8000
Epoch 5/20
 - 0s - loss: 0.3921 - acc: 0.8250 - val_loss: 0.3151 - val_acc: 0.7667
Epoch 6/20
 - 0s - loss: 0.4361 - acc: 0.7833 - val_loss: 0.2431 - val_acc: 0.9667
Epoch 7/20
 - 0s - loss: 0.4025 - acc: 0.8333 - val_loss: 0.2104 - val_acc: 0.9333
Epoch 8/20
 - 0s - loss: 0.2983 - acc: 0.8833 - val_loss: 0.1754 - val_acc: 0.9333
Epoch 9/20
 - 0s - loss: 0.2977 - acc: 0.8667 - val_loss: 0.1591 - val_acc: 0.9333
Epoch 10/20
 - 0s - loss: 0.2948 - acc: 0.8500 - val_loss: 0.1632 - val_acc: 0.9333
Epoch 11/20
 - 0s - loss: 0.2469 - acc: 0.8917 - val_loss: 0.2070 - val_acc: 0.9000
Epoch 12/20
 - 0s - loss: 0.2704 - acc: 

In [207]:
# create the model
model2 = Sequential()

# Dense -> fully connected, (node num, input size, activation function)
model2.add(Dense(64, input_dim=34, activation='relu'))
#input_shape=(784,) 等價於 input_dim=784 ，接收 n 個 784 input
model2.add(Dropout(0.2))
model2.add(Dense(64, activation='relu'))
model2.add(Dropout(0.2))
model2.add(Dense(64, activation='relu'))
model2.add(Dropout(0.2))
model2.add(Dense(64, activation='relu'))
model2.add(Dropout(0.2))
model2.add(Dense(2, activation='softmax'))


# compile the model
#model.compile(loss='categorical_crossentropy',
#              optimizer='sgd',
#              metrics=['accuracy'])            
model2.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])            


# train the model on data
model2.fit(ionosphereX_train, ionosphereY_train,
          batch_size=1, # how many data to look at together
          epochs=20,    # how many epochs to run before stopping
          verbose=2,  # 訓練的詳細資料, 0=安靜, 1=進度條, 2=每輪一行
          validation_data=(ionosphereX_test, ionosphereY_test)
         )

# test the trained model on the testing set
score = model2.evaluate(ionosphereX_train, ionosphereY_train, verbose=0) 
print("\n== ionosphere ==")
print('Train loss:', score[0]) #loss
print('Train accuracy:', score[1])
score = model2.evaluate(ionosphereX_test, ionosphereY_test, verbose=0) 

print('Test loss:', score[0]) #loss
print('Test accuracy:', score[1])

Train on 280 samples, validate on 71 samples
Epoch 1/20
 - 2s - loss: 0.5433 - acc: 0.7250 - val_loss: 0.3326 - val_acc: 0.8873
Epoch 2/20
 - 0s - loss: 0.3277 - acc: 0.9071 - val_loss: 0.2303 - val_acc: 0.9155
Epoch 3/20
 - 0s - loss: 0.2115 - acc: 0.9286 - val_loss: 0.1300 - val_acc: 0.9577
Epoch 4/20
 - 0s - loss: 0.1543 - acc: 0.9500 - val_loss: 0.1103 - val_acc: 0.9437
Epoch 5/20
 - 0s - loss: 0.1610 - acc: 0.9500 - val_loss: 0.1597 - val_acc: 0.9296
Epoch 6/20
 - 0s - loss: 0.1628 - acc: 0.9607 - val_loss: 0.1334 - val_acc: 0.9296
Epoch 7/20
 - 0s - loss: 0.1240 - acc: 0.9750 - val_loss: 0.1475 - val_acc: 0.9577
Epoch 8/20
 - 0s - loss: 0.0831 - acc: 0.9750 - val_loss: 0.2008 - val_acc: 0.9155
Epoch 9/20
 - 0s - loss: 0.0738 - acc: 0.9714 - val_loss: 0.2605 - val_acc: 0.9155
Epoch 10/20
 - 0s - loss: 0.1047 - acc: 0.9714 - val_loss: 0.1396 - val_acc: 0.9155
Epoch 11/20
 - 0s - loss: 0.0801 - acc: 0.9786 - val_loss: 0.1240 - val_acc: 0.9296
Epoch 12/20
 - 0s - loss: 0.1126 - acc: 